# Velocyto 

Clean up velocyto IDs from loom files and combine sample looms into one file 

In [ ]:
import scvelo as scv
import scanpy as sc

import os

import pandas as pd
import numpy as np
import re

import loompy as lp

In [ ]:
os.chdir('/research/peer/fdeckert/FD20200109SPLENO')

# Import velocyto results and convert to AnnData with correct cell id

In [137]:
adata = sc.read_h5ad('data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/raw.h5ad')

In [138]:
# Cell id idx
adata.obs['cell_id_idx'] = [re.search(r"_(.+)", index).group(1) for index in adata.obs.index]

In [139]:
def velocyto_import(x): 
    
    # Import loom 
    adata=sc.read_loom(x['sample_path']+'/velocyto.loom')
    
    # Correct cell id
    cell_id = adata.obs_names
    cell_id=cell_id.str.split(':').str.get(1) # Cut after file name appendix
    cell_id=cell_id.str.split('x').str.get(0) # Remove tailing x
    cell_id=cell_id + '-1_' + str(x['cell_id_idx'])
    adata.obs_names=cell_id
    
    # Make unique names 
    adata.var_names_make_unique()
    
    return(adata)

In [135]:
adata_v = adata.obs[['sample_path', 'cell_id_idx']].drop_duplicates().reset_index(drop=True).apply(velocyto_import, axis=1).to_list()
adata_v = sc.concat(adata_v)

# Store results

In [ ]:
adata_v.write_h5ad('data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/velocyto.h5ad')